In [ ]:
using Plots
using PAM
using FUSE

# Generate inputs (start with D3D template)

In [ ]:

dd_D3D = IMAS.json2imas("/Users/galinaavdeeva/.julia/dev/PAM/examples/template_D3D_1layer_2species.json");

#-------- Check inputs ------------------------
#plot(dd.equilibrium)
#plot(dd_D3D.core_profiles)
 


# Display and modify pellet inputs

In [ ]:
#display(dd_D3D.pellets)

dd_D3D.pellets.time_slice[].pellet[1].velocity_initial = 200.0;


# Running PAM

In [ ]:

inputs=(
   t_start = 0.0,
   t_finish = 0.0005,
   time_step = 0.0001, 
   drift_model= :HPI2,
   Bt_dependance=true,
   update_plasma = false,
)


pellet_PAM = PAM.run_PAM(dd_D3D; inputs...);

# Plot pellet trajectory and density source deposition

In [ ]:
plot(dd_D3D.equilibrium;cx=true)
plot!(pellet_PAM; plot_cloud=false)

# Plot pellet radius, ablation

In [ ]:
plot(pellet_PAM.time, pellet_PAM.radius)

In [ ]:

plot(pellet_PAM.r, pellet_PAM.ablation_rate)

# Compare with D3D OMFIT case

In [ ]:
# results for D3D case DT - 50-50, radius = 0.1 cm, velocity = -200, with Bt dependance and Parks drift model

omfit_radius=[0.1, 0.1, 0.09907301, 0.09514626, 0.08962336, 0.08174432, 0.07028775, 0.05297672, 0.02178495, 0.0 , 0.0,  0.0 , 0.0, 0.0, 0.0];

omfit_R = [2.3,  2.28, 2.26, 2.24, 2.22, 2.2,  2.18, 2.16, 2.14, 2.12, 2.1,  2.08, 2.06, 2.04, 2.02];
omfit_time = [0.,  0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.,  1.1, 1.2, 1.3, 1.4];
omfit_ablation_D = [0.00000000e+00, 0.00000000e+00, 1.15829553e+23, 1.62737315e+23, 2.03252305e+23, 2.43448343e+23, 2.64645468e+23, 2.35555840e+23,
9.11641845e+22, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00];


In [ ]:

plot(pellet_PAM.time, pellet_PAM.radius)
plot!(omfit_time, omfit_radius,markershape=:circle)


In [ ]:
plot(pellet_PAM.r, pellet_PAM.ablation_rate)
plot!(omfit_R, omfit_ablation_D*2,markershape=:circle)

# Use PAM actor

In [ ]:
act = FUSE.ParametersActors();
#start from new dd
dd_D3D = IMAS.json2imas("/Users/galinaavdeeva/.julia/dev/PAM/examples/template_D3D_1layer_2species.json");

# Resize dd to add pellet information to the pulse schedule (requred for the actor)

In [ ]:
resize!(dd_D3D.pulse_schedule.pellet.launcher, 1);
dd_D3D.pulse_schedule.pellet.time = dd_D3D.pellets.time;
dd_D3D.pulse_schedule.pellet.launcher[1].frequency.reference = [2.0];
# also need to add information to the launcher
resize!(dd_D3D.pellets.launcher, 1);
dd_D3D.pellets.launcher[1].name = "pellet";
#save dd for future use
dd_D3D_to_restart=dd_D3D;

In [ ]:
actor = FUSE.ActorPAM(dd_D3D, act);


# Plot density source from the pellet

In [ ]:
plot(dd_D3D.core_sources)

# Use PAM actor to update plasma density

In [ ]:
#show fields of the actor
display(act.ActorPAM)


In [ ]:
# use actor with plasma density update
act.ActorPAM.density_update = true;

In [ ]:
actor = FUSE.ActorPAM(dd_D3D_to_restart, act);
plot(dd_D3D_to_restart.core_profiles)
plot!(dd_D3D.core_profiles)